# API(JSON) 데이터 수집

In [65]:
import urllib.request # urllib = URL 처리를 위한 파이썬 패키지
import json
import mysql.connector
from datetime import datetime

api_key = '432c2ef2c9054447b68d661e93351705'
serviceKey = api_key
startDate = '20150101'
endDate = '20241231'
url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/StockRestService/getSowDataList/{serviceKey}/{startDate}/{endDate}"
request = urllib.request.Request(url) # url 요청
response = json.loads(urllib.request.urlopen(request).read().decode('utf-8'))

In [66]:
data_list = response

In [ ]:
data_list

In [70]:
pigmom_tableqry = """CREATE TABLE pigmom_sow(
    id INT AUTO_INCREMENT PRIMARY KEY,
    farm_id VARCHAR(50),
    sow_id VARCHAR(20),
    parity INT,
    degree INT,
    period_day INT,
    mating_dt DATE,
    pregnancy_day INT,
    pregnancy_nm VARCHAR(50),
    farrow_day FLOAT,
    farrow_dt DATE,
    total_born INT,
    mummy INT,
    still_born INT,
    live_born INT,
    farrow_die INT,
    adt_in VARCHAR(50),
    adt_out VARCHAR(50),
    born_dead INT,
    born_wean INT,
    wean_day FLOAT,
    wean_dt DATE,
    wean_pigs INT
)"""

In [71]:
remote = mysql.connector.connect(
    host = "database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "49841637ok",
    database = "pig"
)

In [72]:
cur = remote.cursor()

In [73]:
cur.execute(pigmom_tableqry)

In [74]:
# 날짜 변환 함수
def parse_date(date_str):
    if date_str:
        try:
            return datetime.strptime(date_str, "%Y%m%d").date()
        except:
            return None
    return None

In [75]:
insert_query = """
    INSERT INTO pigmom_sow(
        farm_id, sow_id, parity, degree, period_day, mating_dt,
        pregnancy_day, pregnancy_nm, farrow_day, farrow_dt,
        total_born, mummy, still_born, live_born, farrow_die,
        adt_in, adt_out, born_dead, born_wean, wean_day,
        wean_dt, wean_pigs
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [76]:
for data in data_list:
    values = (
    data.get('farmId'),
    data.get('sowId'),
    int(float(data['parity'])) if data['parity'] else None,
    int(float(data['degree'])) if data['degree'] else None,
    int(float(data['periodDay'])) if data['periodDay'] else None,
    parse_date(data['matingDt']),
    int(float(data['pregnancyDay'])) if data['pregnancyDay'] else None,
    data.get('pregnancyNm'),
    float(data['farrowDay']) if data['farrowDay'] else None,
    parse_date(data['farrowDt']),
    int(float(data['totalBorn'])) if data['totalBorn'] else None,
    int(float(data['mummy'])) if data['mummy'] else None,
    int(float(data['stillBorn'])) if data['stillBorn'] else None,
    int(float(data['liveBorn'])) if data['liveBorn'] else None,
    int(float(data['farrowDie'])) if data['farrowDie'] else None,
    data.get('adtIn'),
    data.get('adtOut'),
    int(float(data['bornDead'])) if data['bornDead'] else None,
    int(float(data['bornWean'])) if data['bornWean'] else None,
    float(data['weanDay']) if data['weanDay'] else None,
    parse_date(data['weanDt']),
    int(float(data['weanPigs'])) if data['weanPigs'] else None
    )

    cur.execute(insert_query, values)

remote.commit()  # 모든 insert 후에 한 번만 커밋

In [77]:
cur.close()
remote.close()

# csv파일로 수집

In [ ]:
import pandas as pd

# 파일 경로 리스트
file_paths = [
    "../../data/pigmom_data/2015 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2016 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2017 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2018 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2019 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2020 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2021 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2022 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2023 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2024 양돈 모돈 데이터.csv"
]

# 각 파일을 DataFrame으로 읽어서 리스트에 저장
dfs = [pd.read_csv(fp) for fp in file_paths]

# 리스트의 모든 DataFrame을 하나로 합치기
combined_df = pd.concat(dfs, ignore_index=True)

# 결과 확인 (앞부분 5행 출력)
print(combined_df.head())



           농장아이디  개체 구별 번호  산차  교배지수  경과일       교배일  임신 사고기간 임신 사고 원인  임신 기간  \
0  PF_0000347_01         1   1     1    5  20150113      NaN      NaN    116   
1  PF_0000347_01         1   2     1    5  20150602      NaN      NaN    112   
2  PF_0000347_01         3   1     1    5  20150106      NaN      NaN    118   
3  PF_0000347_01         3   2     2    0  20150623      NaN      NaN    116   
4  PF_0000347_01         4   1     1    5  20150113      NaN      NaN    117   

        분만일  ...  사산  실산자수  생시도태두수  양자전입두수  양자전출두수  포유자돈폐사 두수  부분이유 두수  포유기간  \
0  20150509  ...   3    14     NaN     NaN     NaN        NaN      NaN  19.0   
1  20150922  ...   0    14     NaN     NaN     NaN        NaN      NaN  23.0   
2  20150504  ...   0     3     NaN     NaN     NaN        NaN      NaN  24.0   
3  20151017  ...   0    14     NaN     NaN     NaN        NaN      NaN  26.0   
4  20150510  ...   5    12     NaN     NaN     NaN        NaN      NaN  18.0   

          이유일  이유두수  
0  20150528.0  1

In [6]:
combined_df.head()

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
0,PF_0000347_01,1,1,1,5,20150113,NaN,NaN,116,20150509,...,3,14,NaN,NaN,NaN,NaN,NaN,19.0,20150528.0,11.0
1,PF_0000347_01,1,2,1,5,20150602,NaN,NaN,112,20150922,...,0,14,NaN,NaN,NaN,NaN,NaN,23.0,20151015.0,12.0
2,PF_0000347_01,3,1,1,5,20150106,NaN,NaN,118,20150504,...,0,3,NaN,NaN,NaN,NaN,NaN,24.0,20150528.0,12.0
3,PF_0000347_01,3,2,2,0,20150623,NaN,NaN,116,20151017,...,0,14,NaN,NaN,NaN,NaN,NaN,26.0,20151112.0,10.0
4,PF_0000347_01,4,1,1,5,20150113,NaN,NaN,117,20150510,...,5,12,NaN,NaN,NaN,NaN,NaN,18.0,20150528.0,12.0


In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117831 entries, 0 to 117830
Data columns (total 22 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   농장아이디      117831 non-null  object 
 1   개체 구별 번호   117831 non-null  int64  
 2   산차         117831 non-null  int64  
 3   교배지수       117831 non-null  int64  
 4   경과일        117831 non-null  int64  
 5   교배일        117831 non-null  int64  
 6   임신 사고기간    9601 non-null    float64
 7   임신 사고 원인   42 non-null      object 
 8   임신 기간      117831 non-null  int64  
 9   분만일        117831 non-null  int64  
 10  총산자수       117831 non-null  int64  
 11  미라         117831 non-null  int64  
 12  사산         117831 non-null  int64  
 13  실산자수       117831 non-null  int64  
 14  생시도태두수     4908 non-null    float64
 15  양자전입두수     4908 non-null    float64
 16  양자전출두수     4908 non-null    float64
 17  포유자돈폐사 두수  15672 non-null   float64
 18  부분이유 두수    1 non-null       float64
 19  포유기간       112147 non-n

In [28]:
combined_df.describe()

,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 기간,분만일,총산자수,미라,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
count,117818.000000,117818.000000,117818.000000,117818.000000,1.178180e+05,9601.000000,117818.000000,1.178180e+05,117818.000000,117818.00000,117818.000000,117818.000000,4908.000000,4908.000000,4908.000000,15672.000000,1.0,112147.000000,1.121470e+05,112147.000000
mean,432.972474,2.582695,1.176450,82.926412,2.019521e+07,23.351318,116.112462,2.019816e+07,12.804003,0.21276,0.962468,11.628792,0.013040,0.118582,0.112673,2.177450,5.0,25.136874,2.019804e+07,10.620284
std,381.206503,2.393422,0.654795,170.410085,1.598476e+04,4.030328,23.728718,1.571922e+04,3.865102,0.76570,1.634804,3.545767,0.181229,0.718186,0.614128,1.594202,NaN,32.052277,1.565671e+04,2.090335
min,1.000000,0.000000,0.000000,-868.000000,2.012120e+07,0.000000,-1045.000000,2.015010e+07,0.000000,0.00000,-4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.0,-343.000000,2.015012e+07,-21.000000
25%,139.000000,1.000000,1.000000,4.000000,2.019012e+07,21.000000,114.000000,2.019052e+07,11.000000,0.00000,0.000000,10.000000,0.000000,0.000000,0.000000,1.000000,5.0,22.000000,2.019052e+07,10.000000
50%,314.000000,2.000000,1.000000,5.000000,2.020021e+07,25.000000,115.000000,2.020061e+07,13.000000,0.00000,0.000000,12.000000,0.000000,0.000000,0.000000,2.000000,5.0,26.000000,2.020060e+07,11.000000
75%,622.000000,4.000000,1.000000,43.000000,2.021021e+07,25.000000,116.000000,2.021060e+07,15.000000,0.00000,1.000000,14.000000,0.000000,0.000000,0.000000,3.000000,5.0,28.000000,2.021052e+07,12.000000
max,1835.000000,20.000000,11.000000,2825.000000,2.024090e+07,148.000000,2671.000000,2.024122e+07,32.000000,18.00000,25.000000,28.000000,7.000000,12.000000,6.000000,18.000000,5.0,10253.000000,2.051011e+07,46.000000


In [13]:
combined_df[combined_df['산차'] == -1]

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
112489,PF_0023019,1,-1,1,0,20220509,NaN,NaN,115,20220901,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112495,PF_0023019,7,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112496,PF_0023019,8,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112499,PF_0023019,11,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112507,PF_0023019,19,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112510,PF_0023019,22,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112515,PF_0023019,27,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112520,PF_0023019,32,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112522,PF_0023019,34,-1,1,0,20220530,NaN,NaN,115,20220922,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112523,PF_0023019,35,-1,1,0,20220530,NaN,NaN,115,20220922,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
index_to_drop = combined_df[combined_df['산차'] == -1].index

In [18]:
combined_df.drop(index_to_drop, inplace=True)

In [19]:
combined_df

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
0,PF_0000347_01,1,1,1,5,20150113,NaN,NaN,116,20150509,...,3,14,NaN,NaN,NaN,NaN,NaN,19.0,20150528.0,11.0
1,PF_0000347_01,1,2,1,5,20150602,NaN,NaN,112,20150922,...,0,14,NaN,NaN,NaN,NaN,NaN,23.0,20151015.0,12.0
2,PF_0000347_01,3,1,1,5,20150106,NaN,NaN,118,20150504,...,0,3,NaN,NaN,NaN,NaN,NaN,24.0,20150528.0,12.0
3,PF_0000347_01,3,2,2,0,20150623,NaN,NaN,116,20151017,...,0,14,NaN,NaN,NaN,NaN,NaN,26.0,20151112.0,10.0
4,PF_0000347_01,4,1,1,5,20150113,NaN,NaN,117,20150510,...,5,12,NaN,NaN,NaN,NaN,NaN,18.0,20150528.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117826,PF_0023203,84,9,1,0,20240814,NaN,NaN,114,20241206,...,4,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117827,PF_0023203,117,11,1,0,20240724,NaN,NaN,115,20241116,...,3,11,NaN,NaN,NaN,NaN,NaN,29.0,20241215.0,11.0
117828,PF_0023203,118,9,1,0,20240801,NaN,NaN,115,20241124,...,3,13,NaN,NaN,NaN,NaN,NaN,28.0,20241222.0,11.0
117829,PF_0023203,119,9,1,0,20240731,NaN,NaN,115,20241123,...,0,6,NaN,NaN,NaN,NaN,NaN,29.0,20241222.0,11.0


In [20]:
# 필요한 경우, 합친 데이터를 CSV 파일로 저장할 수 있습니다.
combined_df.to_csv("../../data/pigmom_data/combined_data.csv", index=False)